In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, get_linear_schedule_with_warmup, Trainer , TrainingArguments
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_

In [ ]:
!pip install rouge

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.config.list_physical_devices('GPU')

tf.test.is_gpu_available()

True

In [ ]:
df = pd.read_csv("/kaggle/input/merge-df/merge_df.csv")

In [ ]:
df

,Unnamed: 0,File_path,Articles,Summaries
0,0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...
1,1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ..."
2,2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
3,3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...
4,4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g..."
...,...,...,...,...
5444,5444,accidents,HONG KONG — Hundreds of pilot whales that s...,more than 500 rescuers tried frantically to se...
5445,5445,sports,"NICE, France — Rivère accepts the complim...",Signing balotelli was not just a way to garner...
5446,5446,business,FRANKFURT — Germans who never really warmed...,Although there was no evidence of that the bun...
5447,5447,sports,Charles Oakley has strong feelings about compe...,He questioned why any n. b. a. free agent woul...


In [ ]:
df = df.drop(columns="Unnamed: 0")

In [ ]:
df

,File_path,Articles,Summaries
0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...
1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ..."
2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...
4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g..."
...,...,...,...
5444,accidents,HONG KONG — Hundreds of pilot whales that s...,more than 500 rescuers tried frantically to se...
5445,sports,"NICE, France — Rivère accepts the complim...",Signing balotelli was not just a way to garner...
5446,business,FRANKFURT — Germans who never really warmed...,Although there was no evidence of that the bun...
5447,sports,Charles Oakley has strong feelings about compe...,He questioned why any n. b. a. free agent woul...


In [ ]:
filtered_df = df[df['File_path'] == 'entertainment']

In [ ]:
filtered_df

,File_path,Articles,Summaries
1328,entertainment,Super Size Me wins writers' award..Super Size ...,Spurlock was given his award on the same day t...
1329,entertainment,"Mogul Wilson backing UK rap band..Tony Wilson,...","Tony Wilson, the music mogul who established t..."
1330,entertainment,Police praise 'courageous' Ozzy..Rock star Ozz...,"""I could have been badly injured or shot or an..."
1331,entertainment,Eastwood's Baby scoops top Oscars..Clint Eastw...,The boxing drama was named best picture and Ea...
1332,entertainment,Actor Scott is new Bond favourite..Bookmaker W...,Bookmaker William Hill has stopped taking bets...
...,...,...,...
5380,entertainment,WASHINGTON — Senator Jeff Sessions was conf...,late tuesday republicans voted to formally sil...
5400,entertainment,"Was it just a year ago that Katie Holmes, Jenn...",aware that the payoff may be negligible many d...
5405,entertainment,Mother Jones was named magazine of the year on...,also honored were pacific standard for feature...
5408,entertainment,"Damien Chazelle, the writer and director of th...",what was it like to shoot there we were able t...


In [ ]:
def remove_emojis_and_symbols(text):
    # Remove emojis
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)
    # Remove non-alphanumeric symbols
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [ ]:
filtered_df['Articles'] = filtered_df['Articles'].apply(remove_emojis_and_symbols)

C:\Users\chunl\AppData\Local\Temp\ipykernel_3084\2494307040.py:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  filtered_df['Articles'] = filtered_df['Articles'].apply(remove_emojis_and_symbols)


In [ ]:
filtered_df['Summaries'] = filtered_df['Summaries'].apply(remove_emojis_and_symbols)

C:\Users\chunl\AppData\Local\Temp\ipykernel_3084\969203157.py:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  filtered_df['Summaries'] = filtered_df['Summaries'].apply(remove_emojis_and_symbols)


In [ ]:
filtered_df

,File_path,Articles,Summaries
1328,entertainment,Super Size Me wins writers awardSuper Size Me ...,Spurlock was given his award on the same day t...
1329,entertainment,Mogul Wilson backing UK rap bandTony Wilson th...,Tony Wilson the music mogul who established th...
1330,entertainment,Police praise courageous OzzyRock star Ozzy Os...,I could have been badly injured or shot or any...
1331,entertainment,Eastwoods Baby scoops top OscarsClint Eastwood...,The boxing drama was named best picture and Ea...
1332,entertainment,Actor Scott is new Bond favouriteBookmaker Wil...,Bookmaker William Hill has stopped taking bets...
...,...,...,...
5380,entertainment,WASHINGTON Senator Jeff Sessions was confi...,late tuesday republicans voted to formally sil...
5400,entertainment,Was it just a year ago that Katie Holmes Jenni...,aware that the payoff may be negligible many d...
5405,entertainment,Mother Jones was named magazine of the year on...,also honored were pacific standard for feature...
5408,entertainment,Damien Chazelle the writer and director of the...,what was it like to shoot there we were able t...


In [ ]:
! pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=97861e73dd64a475b38d8467229c0eb5cbac6d790c170fcfa59124fa78594b33
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
df= filtered_df.drop(columns = "File_path")

In [ ]:
df.head()

,Articles,Summaries
1328,Super Size Me wins writers' award..Super Size ...,Spurlock was given his award on the same day t...
1329,"Mogul Wilson backing UK rap band..Tony Wilson,...","Tony Wilson, the music mogul who established t..."
1330,Police praise 'courageous' Ozzy..Rock star Ozz...,"""I could have been badly injured or shot or an..."
1331,Eastwood's Baby scoops top Oscars..Clint Eastw...,The boxing drama was named best picture and Ea...
1332,Actor Scott is new Bond favourite..Bookmaker W...,Bookmaker William Hill has stopped taking bets...


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize and preprocess the text data
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
max_length = 1024  # Maximum sequence length

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
X_train = torch.stack([seq.squeeze() for seq in train_df['TokenizedText']])
Y_train = torch.stack([seq.squeeze() for seq in train_df['TokenizedSummary']])
X_test = torch.stack([seq.squeeze() for seq in test_df['TokenizedText']])
Y_test = torch.stack([seq.squeeze() for seq in test_df['TokenizedSummary']])

In [ ]:
train_dataset = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataset = TensorDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [ ]:
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['Articles']]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    targets = [summary for summary in examples['Summaries']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
articles = DatasetDict()

In [ ]:
data_train, data_test = train_test_split(df, test_size=0.10, random_state=42)

In [ ]:
train_ds = Dataset.from_pandas(data_train, preserve_index=False)
test_ds = Dataset.from_pandas(data_test, preserve_index=False)

In [ ]:
articles['train'] = train_ds
articles['test'] = test_ds

In [ ]:
dataset_train = articles['train']
dataset_valid = articles['test']

In [ ]:
from transformers import BartTokenizer

In [ ]:
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=4
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [ ]:
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=4
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [ ]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
training_args = TrainingArguments(
    output_dir="results_bart",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir="results_bart",
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=200,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0003,
    dataloader_num_workers=4
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

In [ ]:
history = trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
200,0.486300,0.195726,0.927000,0.870600,0.915700,173.150500
400,0.373800,0.180362,0.932600,0.876400,0.920200,173.150500
600,0.114400,0.162766,0.939300,0.888200,0.927300,173.150500
800,0.078500,0.145852,0.946600,0.901900,0.936000,173.150500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

In [ ]:
trainer.evaluate()

{'eval_loss': 0.14552676677703857,
 'eval_rouge1': 0.9467,
 'eval_rouge2': 0.9022,
 'eval_rougeL': 0.936,
 'eval_gen_len': 173.1505,
 'eval_runtime': 7.7802,
 'eval_samples_per_second': 11.953,
 'eval_steps_per_second': 3.085,
 'epoch': 4.0}

In [ ]:
trainer.save_model("/kaggle/working/bart-base-entertainment-summarizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
model.save_pretrained("/kaggle/working/bart-base-summarizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
#optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=50, num_training_steps=len(train_dataloader) * 10)  # Add learning rate scheduler
#early_stopping_rounds = 2
#best_rouge_score = -1
#current_round = 0

In [ ]:
#for epoch in range(3):
    #train_loss = train(model, train_dataloader, optimizer, scheduler)
    #print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

Training: 100%|██████████| 185/185 [06:16<00:00,  2.04s/it]


Epoch 1, Train Loss: 0.7028


Training: 100%|██████████| 185/185 [06:28<00:00,  2.10s/it]


Epoch 2, Train Loss: 0.7044


Training: 100%|██████████| 185/185 [06:28<00:00,  2.10s/it]

Epoch 3, Train Loss: 0.7034


In [ ]:
abc = "Superhero clichés: The 45-year-old actor and stand up comedian played an alien who came to Earth to protect his planet, becoming a Bollywood star to blend in. But despite high anticipation for the film, which was directed by Oscar winner Chloé Zhao, it received a relatively poor 47% rating on Rotten Tomatoes and achieved the lowest score of any Marvel Cinematic Universe film in audience survey Cinema Score. Empire said at the time of release: Chloé Zhao's entry into the superhero world is assured, ambitious and told on a dizzyingly cosmic scale - but even it can't escape the clichés of superhero storytelling. The Guardian's Peter Bradshaw wrote: There are some nice touches and an attractive new diversity worn lightly, but this is an underpowered and uncertain film. Nanjiani and his wife, Emily V Gordon, were Oscar nominees in 2018 for their romantic comedy The Big Sick, in which he starred alongside Zoe Kazan. He's one of many stars who have spoken out about dealing with bad reviews in film and TV. Melissa McCarthy said in an interview that she confronted a critic at the Toronto Film Festival who had previously criticised her for her appearance in 2014 film Tammy. Jennifer Lawrence said in Variety that she gets defensive after reading negative reviews of her work. You're so in the zone, you put your whole soul and body, you move to shoot a movie, and you then love it, obviously because you wouldn't be there if you didn't love it, and then people just destroy it, she explained."

In [ ]:
abc

"Superhero clichés: The 45-year-old actor and stand up comedian played an alien who came to Earth to protect his planet, becoming a Bollywood star to blend in. But despite high anticipation for the film, which was directed by Oscar winner Chloé Zhao, it received a relatively poor 47% rating on Rotten Tomatoes and achieved the lowest score of any Marvel Cinematic Universe film in audience survey Cinema Score. Empire said at the time of release: Chloé Zhao's entry into the superhero world is assured, ambitious and told on a dizzyingly cosmic scale - but even it can't escape the clichés of superhero storytelling. The Guardian's Peter Bradshaw wrote: There are some nice touches and an attractive new diversity worn lightly, but this is an underpowered and uncertain film. Nanjiani and his wife, Emily V Gordon, were Oscar nominees in 2018 for their romantic comedy The Big Sick, in which he starred alongside Zoe Kazan. He's one of many stars who have spoken out about dealing with bad reviews i

In [ ]:
inputs = tokenizer(abc, return_tensors='pt', max_length=1024, truncation=True)


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Define your article


# Initialize BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Tokenize the article
inputs = tokenizer(abc, return_tensors='pt', max_length=1024, truncation=True)

# Generate predictions
outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)
predicted_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicted Summary:", predicted_summary)

Predicted Summary: Superhero clichés: The 45-year-old actor and stand up comedian played an alien who came to Earth to protect his planet, becoming a Bollywood star to blend in. But despite high anticipation for the film, which was directed by Oscar winner Chloé Zhao, it received a relatively poor 47% rating on Rotten Tomatoes and achieved the lowest score of any Marvel Cinematic Universe film in audience survey Cinema Score. Empire said at the time of release: Chloek Zhao's entry into the superhero world is assured, ambitious and told on a dizzyingly cosmic scale - but even it can't escape the clichés of superhero storytelling. The Guardian's Peter Bradshaw wrote: There are some nice touches and an attractive new diversity worn lightly, but this is an underpowered and uncertain film. Nanjiani and his wife, Emily V Gordon, were Oscar nominees in 2018 for their romantic comedy The Big Sick, in which he starred alongside Zoe Kazan. He's one of many stars who have spoken out about dealing

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Tokenize the article
inputs = tokenizer(abc, return_tensors='pt', max_length=1024, truncation=True)

# Generate predictions
outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)
predicted_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicted Summary:", predicted_summary)

Predicted Summary: Superhero clichés: The 45-year-old actor and stand up comedian played an alien who came to Earth to protect his planet, becoming a Bollywood star to blend in. But despite high anticipation for the film, which was directed by Oscar winner Chloé Zhao, it received a relatively poor 47% rating on Rotten Tomatoes and achieved the lowest score of any Marvel Cinematic Universe film in audience survey Cinema Score. Empire said at the time of release: Chloek Zhao's entry into the superhero world is assured, ambitious and told on a dizzyingly cosmic scale - but even it can't escape the clichés of superhero storytelling. The Guardian's Peter Bradshaw wrote: There are some nice touches and an attractive new diversity worn lightly, but this is an underpowered and uncertain film. Nanjiani and his wife, Emily V Gordon, were Oscar nominees in 2018 for their romantic comedy The Big Sick, in which he starred alongside Zoe Kazan. He's one of many stars who have spoken out about dealing